In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../utils"))
from aoc_utils import load_data, check

In [ ]:
from functools import cache

In [ ]:
data = load_data(2023, 12)

In [ ]:
# data, part_1, part_2
tests = [
    (
        """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""",
        21,
        525152,
    ),
]

# Part 1

In [ ]:
@cache
def line_arrangements(line, clues):
    if len(clues) == 0:
        if "#" in line:
            return 0
        return 1
    if sum(clues) + len(clues) - 1 > len(line):
        return 0

    start = 0
    c = clues[0]
    while (
        "." in line[start:start + c]
        or (start + c < len(line) and line[start + c] == "#")
        or (start > 0 and line[start - 1] == "#")
    ):
        if line[start] == "#" or start + c >= len(line):
            return 0
        start += 1
    ret = line_arrangements(line[start + c + 1:], clues[1:])
    if line[start] == "?":
        ret += line_arrangements(line[start + 1:], clues)
    return ret

In [ ]:
def arrangements(data, repeat=1):
    arr = 0
    for line in data.splitlines():
        springs, clues_str = line.split()
        clues = tuple(int(v) for v in clues_str.split(","))
        arr += line_arrangements("?".join([springs] * repeat), clues * repeat)
    return arr

In [ ]:
check(arrangements, tests)
arrangements(data)

# Part 2

In [ ]:
check(arrangements, tests, 2, repeat=5)
arrangements(data, repeat=5)